In [1]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q timm torchmetrics albumentations
!pip install -q mlflow dagshub dvc
!pip install -q python-dotenv pyyaml

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.2 which is incompatible.
gradio 5.10.0 requires aiofiles<24.0,>=22.0, but you have aiofiles 24.1.0 which is incompatible.
langchain-chroma 0.2.1 requires numpy<2.0.0,>=1.22.4; python_version < "3.12", but you have numpy 2.2.6 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
unstructured 0.16.23 requires numpy<2, but you have numpy 2.2.6 which is incompatible.

[notice] A ne

In [6]:
%pip install -q --upgrade numpy scipy scikit-learn
%pip install --force-reinstall --no-cache-dir numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.4.1 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.4.1 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.17.0 which is incompatible.
gradio 5.10.0 requires aiofiles<24.0,>=22.0, but you have aiofiles 24.1.0 which is incompatible.
langchain-chroma 0.2.1 requires numpy<2.0.0,>=1.22.4; python_version < "3.12", but you have numpy 2.4.1 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but y

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.6 MB 10.0 MB/s eta 0:00:02
   -------------- ------------------------- 4.7/12.6 MB 14.2 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.6 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.1
    Uninstalling numpy-2.4.1:
      Successfully uninstalled numpy-2.4.1


In [8]:
import os
import sys
import json
import shutil
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score
)
from sklearn.preprocessing import LabelEncoder

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import timm
import torchmetrics
from torchmetrics import Accuracy, Precision, Recall, F1Score

import mlflow
import mlflow.pytorch
import dagshub

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_insta

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [1]:
CONFIG = {
    "DAGSHUB_USERNAME":"shashwatsingh0511",
    "DAGSHUB_TOKEN":"99c593c5d3e7fd5f7db3c0e90e70ca536fcd8a7e",
    "DAGSHUB_REPO":"radae_mlops",
    
    # Data
    "DATA_DIR": "data\raw",
    "OUTPUT_DIR": "data\processed",
    
    # Model
    "NUM_CLASSES": 5,
    "IMAGE_SIZE": 224,
    "BACKBONE": "efficientnet_b0",
    
    # Training
    "EPOCHS": 30,
    "BATCH_SIZE": 32,
    "LEARNING_RATE": 1e-4,
    "WEIGHT_DECAY": 1e-4,
    
    # Classes
    'CLASS_MAPPING': {
        '0': 'bicycle',
        '1': 'car',
        '2': '1_person',
        '3': '2_persons',
        '4': '3_persons',
        '5': 'mixed_car_bike_person',
        '6': 'person_bicycle',
        '7': 'person_car',
        '8': 'bicycle_car'
    }
    

}


print("COnfig loaded")

for k,v in CONFIG.items():
    if "TOKEN" not in k:
        print(f"{k}: {v}")

COnfig loaded
DAGSHUB_USERNAME: shashwatsingh0511
DAGSHUB_REPO: radae_mlops
awTA_DIR: data
OUTPUT_DIR: data\processed
NUM_CLASSES: 5
IMAGE_SIZE: 224
BACKBONE: efficientnet_b0
EPOCHS: 30
BATCH_SIZE: 32
LEARNING_RATE: 0.0001
WEIGHT_DECAY: 0.0001
CLASS_MAPPING: {'0': 'bicycle', '1': 'car', '2': '1_person', '3': '2_persons', '4': '3_persons', '5': 'mixed_car_bike_person', '6': 'person_bicycle', '7': 'person_car', '8': 'bicycle_car'}
